This is a separate file for when I tried one hot encoding. I did not like the results so I let it be.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

#Dataset Info 
# 1- Student Age (1: 18-21, 2: 22-25, 3: above 26)
# 2- Sex (1: female, 2: male)
# 3- Graduated high-school type: (1: private, 2: state, 3: other)
# 4- Scholarship type: (1: None, 2: 25%, 3: 50%, 4: 75%, 5: Full)
# 5- Additional work: (1: Yes, 2: No)
# 6- Regular artistic or sports activity: (1: Yes, 2: No)
# 7- Do you have a partner: (1: Yes, 2: No)
# 8- Total salary if available (1: USD 135-200, 2: USD 201-270, 3: USD 271-340, 4: USD 341-410, 5: above 410)
# 9- Transportation to the university: (1: Bus, 2: Private car/taxi, 3: bicycle, 4: Other)
# 10- Accommodation type in Cyprus: (1: rental, 2: dormitory, 3: with family, 4: Other)
# 11- Mothersâ€™ education: (1: primary school, 2: secondary school, 3: high school, 4: university, 5: MSc., 6: Ph.D.)
# 12- Fathersâ€™ education: (1: primary school, 2: secondary school, 3: high school, 4: university, 5: MSc., 6: Ph.D.)
# 13- Number of sisters/brothers (if available): (1: 1, 2:, 2, 3: 3, 4: 4, 5: 5 or above)
# 14- Parental status: (1: married, 2: divorced, 3: died - one of them or both)
# 15- Mothersâ€™ occupation: (1: retired, 2: housewife, 3: government officer, 4: private sector employee, 5: self-employment, 6: other)
# 16- Fathersâ€™ occupation: (1: retired, 2: government officer, 3: private sector employee, 4: self-employment, 5: other)
# 17- Weekly study hours: (1: None, 2: <5 hours, 3: 6-10 hours, 4: 11-20 hours, 5: more than 20 hours)
# 18- Reading frequency (non-scientific books/journals): (1: None, 2: Sometimes, 3: Often)
# 19- Reading frequency (scientific books/journals): (1: None, 2: Sometimes, 3: Often)
# 20- Attendance to the seminars/conferences related to the department: (1: Yes, 2: No)
# 21- Impact of your projects/activities on your success: (1: positive, 2: negative, 3: neutral)
# 22- Attendance to classes (1: always, 2: sometimes, 3: never)
# 23- Preparation to midterm exams 1: (1: alone, 2: with friends, 3: not applicable)
# 24- Preparation to midterm exams 2: (1: closest date to the exam, 2: regularly during the semester, 3: never)
# 25- Taking notes in classes: (1: never, 2: sometimes, 3: always)
# 26- Listening in classes: (1: never, 2: sometimes, 3: always)
# 27- Discussion improves my interest and success in the course: (1: never, 2: sometimes, 3: always)
# 28- Flip-classroom: (1: not useful, 2: useful, 3: not applicable)
# 29- Cumulative grade point average in the last semester (/4.00): (1: <2.00, 2: 2.00-2.49, 3: 2.50-2.99, 4: 3.00-3.49, 5: above 3.49)
# 30- Expected Cumulative grade point average in the graduation (/4.00): (1: <2.00, 2: 2.00-2.49, 3: 2.50-2.99, 4: 3.00-3.49, 5: above 3.49)
# 31- Course ID
# 32- OUTPUT Grade (0: Fail, 1: DD, 2: DC, 3: CC, 4: CB, 5: BB, 6: BA, 7: AA)

data_set = pd.read_csv('StudentsPerformance_with_headers.csv')
df = data_set.copy()
#dropping course ID and student ID because they are not relevant
df.drop('COURSE ID', axis=1, inplace=True)
df.drop('STUDENT ID', axis=1, inplace=True)

one_hot_df = pd.get_dummies(df, columns = ['Student Age','Sex','Graduated high-school type', 'Scholarship type','Additional work','Regular artistic or sports activity','Do you have a partner','Total salary if available','Transportation to the university','Accommodation type in Cyprus','Mother’s education','Father’s education ','Number of sisters/brothers','Parental status','Mother’s occupation','Father’s occupation','Weekly study hours','Reading frequency','Reading frequency.1','Attendance to the seminars/conferences related to the department','Impact of your projects/activities on your success','Attendance to classes','Preparation to midterm exams 1','Preparation to midterm exams 2','Taking notes in classes','Listening in classes','Discussion improves my interest and success in the course','Flip-classroom','Cumulative grade point average in the last semester (/4.00)','Expected Cumulative grade point average in the graduation (/4.00)']) 
print(one_hot_df)

     GRADE  Student Age_1  Student Age_2  Student Age_3  Sex_1  Sex_2  \
0        1              0              1              0      0      1   
1        1              0              1              0      0      1   
2        1              0              1              0      0      1   
3        1              1              0              0      1      0   
4        1              0              1              0      0      1   
..     ...            ...            ...            ...    ...    ...   
140      5              0              1              0      1      0   
141      5              1              0              0      1      0   
142      1              1              0              0      1      0   
143      4              0              1              0      1      0   
144      3              1              0              0      1      0   

     Graduated high-school type_1  Graduated high-school type_2  \
0                               0                       

In [21]:
# Random Forest Classifier run
print("Random Forest Classier Model")

def calculate_adjacent_agreement(true_labels, predicted_labels):
    adjacent_agreement = sum(abs(true_labels.iloc[i] - predicted_labels[i]) <= 1 and
                             abs(true_labels.iloc[i + 1] - predicted_labels[i + 1]) <= 1
                             for i in range(len(true_labels) - 1)) / (len(true_labels) - 1)
    return adjacent_agreement

numeric_df = one_hot_df.select_dtypes(include=['number']).dropna()

X = numeric_df.drop('GRADE', axis=1)  
y = numeric_df['GRADE']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a RandomForestRegressor instance
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

#Train
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

#add adjacent agreement- pass in true label and predicted if absolute value of difference is 1 or less return 1 otherwise 0
adjacent_agreement = calculate_adjacent_agreement(y_test, y_pred)
print(f"\nAdjacent Agreement: {adjacent_agreement:.4f}")

Random Forest Classier Model
Accuracy: 0.2500

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.22      0.60      0.32        10
           2       0.00      0.00      0.00         9
           3       0.50      0.67      0.57         6
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         5
           7       0.20      0.33      0.25         3

    accuracy                           0.25        44
   macro avg       0.12      0.20      0.14        44
weighted avg       0.13      0.25      0.17        44


Confusion Matrix:
[[0 3 1 0 0 0 0 1]
 [0 6 2 0 0 0 0 2]
 [0 9 0 0 0 0 0 0]
 [0 1 1 4 0 0 0 0]
 [0 2 0 0 0 0 0 1]
 [0 2 0 1 0 0 0 0]
 [0 3 0 2 0 0 0 0]
 [0 1 0 1 0 0 0 1]]

Adjacent Agreement: 0.3953


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
#random forest regressor run
print("Random Forest Regressor Model")
numeric_df = one_hot_df.select_dtypes(include=['number']).dropna()

X = numeric_df.drop('GRADE', axis=1)  
y = numeric_df['GRADE']

#split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#random forest regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

#train
rf_regressor.fit(X_train, y_train)

# Make predictions on test set
y_pred = rf_regressor.predict(X_test)

# Evaluate model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"\nRoot Mean Squared Error (RMSE): {rmse}")

# Print feature importances
feature_importances = rf_regressor.feature_importances_
print("\nFeature Importances:")
for col, importance in sorted(zip(X.columns, feature_importances), key=lambda x: x[1], reverse=True):
    print(f"{col}: {importance:.4f}")
    
correlations_pearson = []
for col in X.columns:
    correlation, _ = pearsonr(X[col], pd.factorize(y)[0])
    correlations_pearson.append((col, correlation))

# Print Pearson correlations
print("\nPearson Correlations:")
for col, correlation in sorted(correlations_pearson, key=lambda x: abs(x[1]), reverse=True):
    print(f"{col}: {correlation:.4f}")

Random Forest Regressor Model

Root Mean Squared Error (RMSE): 2.010543797990077

Feature Importances:
Scholarship type_4: 0.0632
Sex_1: 0.0457
Sex_2: 0.0422
Total salary if available_1: 0.0276
Flip-classroom_1: 0.0253
Flip-classroom_2: 0.0250
Mother’s education_3: 0.0249
Cumulative grade point average in the last semester (/4.00)_2: 0.0237
Regular artistic or sports activity_1: 0.0214
Number of sisters/brothers_5: 0.0212
Cumulative grade point average in the last semester (/4.00)_5: 0.0205
Father’s occupation_2: 0.0198
Student Age_1: 0.0197
Father’s education _3: 0.0186
Mother’s occupation_2: 0.0183
Regular artistic or sports activity_2: 0.0174
Cumulative grade point average in the last semester (/4.00)_1: 0.0161
Father’s education _4: 0.0159
Cumulative grade point average in the last semester (/4.00)_4: 0.0159
Preparation to midterm exams 1_3: 0.0133
Number of sisters/brothers_3: 0.0129
Reading frequency_1: 0.0128
Mother’s occupation_4: 0.0126
Weekly study hours_1: 0.0118
Accommodati